In [1]:
# !git clone https://github.com/soCzech/TransNet

Cloning into 'TransNet'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 27 (delta 0), reused 0 (delta 0), pack-reused 24
Unpacking objects: 100% (27/27), done.


In [ ]:
# !pip3 install --upgrade --force-reinstall tensorflow-gpu==1.15.0 
# !pip install protobuf==3.20.*
# !pip install ffmpeg-python

In [ ]:
# !gdown --id 1bcAds2qz0rLDvwSCXFmaNozHWVu1DKj6

In [1]:
%cd TransNet

/media/nhattuong/Data/TranseNet/TransNet


In [2]:
import ffmpeg
import numpy as np
import tensorflow as tf

from transnet import TransNetParams, TransNet
from transnet_utils import draw_video_with_predictions, scenes_from_predictions

import glob
import os
import cv2

In [3]:
# initialize the network
params = TransNetParams()
params.CHECKPOINT_PATH = "./model/transnet_model-F16_L3_S2_D256"

net = TransNet(params)


[TransNet] Creating ops.


           Input (?, ?, 27, 48, 3)
           SDDCNN_1
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


2022-11-11 02:08:50.424538: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-11-11 02:08:50.445043: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-11 02:08:50.445151: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: NVIDIA GeForce RTX 3090 Ti major: 8 minor: 6 memoryClockRate(GHz): 1.95
pciBusID: 0000:01:00.0
2022-11-11 02:08:50.445226: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcudart.so.10.0'; dlerror: libcudart.so.10.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/nhattuong/anaconda3/envs/transnet/lib/python3.7/site-packages/cv2/../../lib64:/usr/local/cuda-11.3/lib64
2022-11-11 02:08:50.445261: W tensorflow/stream_executor/plat

           > DDCNN_1 (?, ?, 27, 48, 64)
           > DDCNN_2 (?, ?, 27, 48, 64)
           MaxPool (?, ?, 13, 24, 64)
           SDDCNN_2
           > DDCNN_1 (?, ?, 13, 24, 128)
           > DDCNN_2 (?, ?, 13, 24, 128)
           MaxPool (?, ?, 6, 12, 128)
           SDDCNN_3
           > DDCNN_1 (?, ?, 6, 12, 256)
           > DDCNN_2 (?, ?, 6, 12, 256)
           MaxPool (?, ?, 3, 6, 256)
           Flatten (?, ?, 4608)
           Dense (?, ?, 256)
           Logits (?, ?, 2)
           Predictions (?, ?)
[TransNet] Network built.

[TransNet] Found 4614850 trainable parameters.

INFO:tensorflow:Restoring parameters from ./model/transnet_model-F16_L3_S2_D256
[TransNet] Parameters restored from 'transnet_model-F16_L3_S2_D256'.


In [4]:
################# Config Folder #################
video_paths = sorted(glob.glob('/media/nhattuong/Data/Dataset_BTC/Videos_Batch3/VideoC02_V02/*.mp4'))
des_path = '/media/nhattuong/Data/TranseNet/TransNet_Database/KeyFramesC02_V02/'
#################################################

# os.makedirs(des_path, exist_ok=True)

for video_path in video_paths:
    folder_name = video_path.split('/')[-1].replace( '.mp4','')

    test_folder = int(folder_name.replace('C02_V',''))
    print(test_folder)
    # if test_folder <= 244 or test_folder==253 or test_folder==271:
    #     print(f"Skip {folder_name}")
    #     continue

    folder_path = des_path + f'{folder_name}'
    os.mkdir(folder_path)

    # export video into numpy array using ffmpeg
    video_stream, err = (
        ffmpeg
        .input(video_path)
        .output('pipe:', format='rawvideo', pix_fmt='rgb24', s='{}x{}'.format(params.INPUT_WIDTH, params.INPUT_HEIGHT))
        .run(capture_stdout=True)
    )
    video = np.frombuffer(video_stream, np.uint8).reshape([-1, params.INPUT_HEIGHT, params.INPUT_WIDTH, 3])

    # predict transitions using the neural network
    predictions = net.predict_video(video)

    # Generate list of scenes from predictions, returns tuples of (start frame, end frame)
    scenes = scenes_from_predictions(predictions, threshold=0.1)
    with open(f"{folder_path}.txt", 'w') as f:
        for sc in scenes:
            f.write(str(sc) + '\n')

    cam = cv2.VideoCapture(video_path)
    currentframe = 0
    index = 0

    while True:
        ret,frame = cam.read() 
        if ret:
            currentframe += 1
            # for sc in scenes:                        
            idx_first = int(scenes[index][0])
            idx_end = int(scenes[index][1])
            idx_025 = int(scenes[index][0] + (scenes[index][1]-scenes[index][0])/4)
            idx_05 = int(scenes[index][0] + (scenes[index][1]-scenes[index][0])/2)
            idx_075 = int(scenes[index][0] + 3*(scenes[index][1]-scenes[index][0])/4)

            #### First #### 
            if currentframe - 1 == idx_first:
                filename_first = "{}/{:0>6d}.jpg".format(folder_path, idx_first)
                # video_save = cv2.resize(video[idx_first], (1280,720))
                cv2.imwrite(filename_first, frame)

            # #### End ####          
            if currentframe - 1 == idx_end:
                filename_end = "{}/{:0>6d}.jpg".format(folder_path, idx_end)
                # video_save = cv2.resize(video[idx_end], (1280,720))
                cv2.imwrite(filename_end, frame)
                index += 1

            #### 025 ####
            if currentframe - 1 == idx_025:
                filename_025 = "{}/{:0>6d}.jpg".format(folder_path, idx_025)
                # video_save = cv2.resize(video[idx_025], (1280,720))
                cv2.imwrite(filename_025, frame)

            # #### 05 ####
            if currentframe - 1 == idx_05:
                filename_05 = "{}/{:0>6d}.jpg".format(folder_path, idx_05)
                # video_save = cv2.resize(video[idx_05], (1280,720))
                cv2.imwrite(filename_05, frame)

            # #### 075 ####
            if currentframe - 1 == idx_075:
                filename_075 = "{}/{:0>6d}.jpg".format(folder_path, idx_075)
                # video_save = cv2.resize(video[idx_075], (1280,720))
                cv2.imwrite(filename_075, frame)

        else:
            break

    cam.release()
    cv2.destroyAllWindows()

Skip C02_V0200
Skip C02_V0201
Skip C02_V0202
Skip C02_V0203
Skip C02_V0204
Skip C02_V0205
Skip C02_V0206
Skip C02_V0207
Skip C02_V0208
Skip C02_V0209
Skip C02_V0210
Skip C02_V0211
Skip C02_V0212
Skip C02_V0213
Skip C02_V0214
Skip C02_V0215
Skip C02_V0216
Skip C02_V0217
Skip C02_V0218
Skip C02_V0219
Skip C02_V0220
Skip C02_V0221
Skip C02_V0222
Skip C02_V0223
Skip C02_V0224
Skip C02_V0225
Skip C02_V0226
Skip C02_V0227
Skip C02_V0228
Skip C02_V0229
Skip C02_V0230
Skip C02_V0231
Skip C02_V0232
Skip C02_V0233
Skip C02_V0234
Skip C02_V0235
Skip C02_V0236
Skip C02_V0237
Skip C02_V0238
Skip C02_V0239
Skip C02_V0240
Skip C02_V0241
Skip C02_V0242
Skip C02_V0243
Skip C02_V0244
246


ffmpeg version 3.4.11-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-li

[TransNet] Processing video frames 84600/84600
247


ffmpeg version 3.4.11-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-li

[TransNet] Processing video frames 98632/98632
248


...

[TransNet] Processing video frames 92701/92701
298


ffmpeg version 3.4.11-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-li

[TransNet] Processing video frames 195150/195150
299


ffmpeg version 3.4.11-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-li

[TransNet] Processing video frames 99152/99152


In [ ]:
# For ilustration purposes, we show only 200 frames starting with frame number 8000.
# draw_video_with_predictions(video[8000:8200], predictions[8000:8200], threshold=0.1)

In [ ]:
# draw_video_with_predictions(video[:], predictions[:], threshold=0.1)

In [ ]:
# Generate list of scenes from predictions, returns tuples of (start frame, end frame)
scenes = scenes_from_predictions(predictions, threshold=0.1)

# For ilustration purposes, only the visualized scenes are shown.
# scenes